In [43]:
from PIL import Image
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel, SegformerImageProcessor, AutoModelForSemanticSegmentation , AutoFeatureExtractor
from urllib.parse import urlparse, urlunparse
from weaviate.util import generate_uuid5
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import weaviate
import numpy as np
from numpy.linalg import norm
import json
import cv2
import base64
from time import sleep
import ast
import re

In [2]:
client = weaviate.Client(url="http://localhost:8080")

In [9]:
customer = open("customer.json", "r")
customer_class = json.load(customer)
customer.close()
# client.schema.delete_class("Customer_Profile")
client.schema.create_class(customer_class)

In [12]:
checkpoint = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(checkpoint)
processor = CLIPProcessor.from_pretrained(checkpoint)

def getTextEmbeddings(text):
	inputs = processor(text=text , images=Image.new('RGB' , (72 , 72)), return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)
	return outputs["text_embeds"]

In [5]:
PATH = "D:\\Codes\\ziggy-ai\\scraping\\Flipkart"

In [11]:
df1 = pd.read_csv(f"{PATH}\\Girls Tops 1.csv")
df2 = pd.read_csv(f"{PATH}\\Girls Tops 2.csv")
df3 = pd.read_csv(f"{PATH}\\Girls Bottoms.csv")
df4 = pd.read_csv(f"{PATH}\\Formal Shirts.csv")
df5 = pd.read_csv(f"{PATH}\\Formal Pants.csv")

In [8]:
user_profile1_top = []
user_profile2_top = []
user_profile3_top = []
user_profile4_top = []
user_profile1_bottom = []
user_profile2_bottom = []
user_profile3_bottom = []
user_profile4_bottom = []

In [15]:
def clean_url(url):
    parsed_url = urlparse(url)
    cleaned_parsed_url = parsed_url._replace(query='')
    cleaned_url = urlunparse(cleaned_parsed_url)
    return cleaned_url

def extract_first_amount(text):
    amount_match = re.search(r'₹(\d+)', text)
    if amount_match:
        first_amount = amount_match.group(1)
        return int(first_amount)
    else:
        return None

def get_rating_and_number(string1, string2):
    rating = string1[0:(len(string1)-len(string2))]
    ratings_match = re.search(r'([\d,.]+) ratings', string2)
    if ratings_match:
        num_ratings = int(ratings_match.group(1).replace(',', ''))
    else:
        num_ratings = 0
    return rating, num_ratings

def get_product_details(dict_string):
    dict = ast.literal_eval(dict_string)
    keys_to_check = ['Color', 'Pattern', 'Fabric', 'Fit', 'Type']
    new_dict = {key: dict.get(key, None) for key in keys_to_check}
    return new_dict

In [40]:
def mean_pooling(vector_list):
    vectors_array = np.array(vector_list)
    mean_pooled_vector = np.mean(vectors_array, axis=0)
    return mean_pooled_vector

In [20]:
def embed_product_details(product):
    name = product['Name']
    dict_string = get_product_details(product['Specifications'])
    colour = dict_string['Color']
    pattern = dict_string['Pattern']
    fabric = dict_string['Fabric']
    fit = dict_string['Fit']
    type = dict_string['Type']
    embedding = getTextEmbeddings(f"Name: {name}, Color: {colour}, Pattern: {pattern}, Fabric: {fabric}, Fit: {fit}, Type: {type}")
    return embedding.tolist()[0]

In [38]:
list_of_products = [0, 3, 4, 5, 8, 9]

In [39]:
for i in list_of_products:
    user_profile4_bottom.append(embed_product_details(df5.iloc[i]))

In [42]:
mean_pooling(user_profile1_top).tolist()

[-0.015457322122529149,
 -0.004500559841593106,
 0.0943158008158207,
 -0.007839408703148365,
 0.051852934062480927,
 0.020159307110588998,
 -0.047120802104473114,
 0.009433484015365442,
 0.018930845777504146,
 -0.019049969579403598,
 -0.05424157405893008,
 -0.061064387361208596,
 0.021112844347953796,
 0.047375814678768315,
 -0.01590080667907993,
 0.04371071420609951,
 0.06940511117378871,
 -0.018135233627011377,
 -0.028368829749524593,
 -0.005592237537105878,
 -0.030445727209250133,
 0.009500534118463596,
 -0.007981105034559732,
 0.0059693421159560485,
 -0.01262392041583856,
 -0.003503641734520594,
 0.02106676607703169,
 0.04046731193860372,
 -0.026347796743114788,
 -0.024223001052935917,
 -0.05929394718259573,
 0.029600038814047974,
 0.008652475119257966,
 0.02983373838166396,
 0.03318018217881521,
 -0.03864450380206108,
 -0.0030259081783394017,
 -0.0014880454012503226,
 -0.02098883253832658,
 0.002104888359705607,
 -0.0017922863674660523,
 -0.0493728124226133,
 -0.09351445206751426,

In [51]:
client.data_object.create(
    {
        "name":"Ron Weasley",
        "gender":"Male",
        "age":"30",
        "category":"Bottom"
    },
    class_name="Customer_Profile",
    uuid=generate_uuid5("Male Profile 2 Bottom"),
    vector=mean_pooling(user_profile4_bottom).tolist()
)

'5ca3dcb3-b211-5e96-ae03-bfec6c1344a9'